In [20]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import os

In [17]:
def normalizar_hoja(ruta_imagen):
    # 1. Cargar la imagen
    img_bgr = cv2.imread(ruta_imagen)
    if img_bgr is None:
        print("Error: No se pudo cargar la imagen.")
        return

    # Convertir a RGB para Matplotlib (OpenCV usa BGR por defecto)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    
    # 2. Pre-procesamiento (Escala de grises)
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    
    # 3. Umbralización (Asumiendo fondo CLARO/BLANCO en tu maqueta)
    # THRESH_BINARY_INV: Vuelve el fondo negro (0) y la hoja blanca (255)
    # Ajusta el '200' según la iluminación de tu maqueta (0-255)
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    
    # 4. Encontrar contornos
    contornos, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contornos:
        print("¡Alerta! No se detectó ninguna hoja (¿Fondo mal iluminado?).")
        return

    # Tomar el contorno más grande (la hoja)
    c = max(contornos, key=cv2.contourArea)
    
    # 5. Calcular el Bounding Box
    x, y, w, h = cv2.boundingRect(c)
    
    # --- VISUALIZACIÓN A: Dibujar la caja en la imagen original ---
    img_con_caja = img_rgb.copy()
    # Dibujar rectángulo Verde (0, 255, 0), grosor 3
    cv2.rectangle(img_con_caja, (x, y), (x+w, y+h), (0, 255, 0), 5)
    
    # --- VISUALIZACIÓN B: El Recorte (Lo que va a la IA) ---
    # Margen de seguridad (padding)
    pad = 20
    h_img, w_img, _ = img_rgb.shape
    y1, y2 = max(0, y-pad), min(h_img, y+h+pad)
    x1, x2 = max(0, x-pad), min(w_img, x+w+pad)
    
    roi = img_rgb[y1:y2, x1:x2]
    
    # Redimensionar a lo que pide MobileNet (224x224)
    roi_final = cv2.resize(roi, (224, 224))

    # --- PLOTEAR CON MATPLOTLIB ---
    plt.figure(figsize=(15, 5))

    # Subplot 1: La Máscara (Lo que ve el algoritmo matemático)
    plt.subplot(1, 3, 1)
    plt.title("1. Máscara Binaria (Threshold)")
    plt.imshow(thresh, cmap='gray')
    plt.axis('off')

    # Subplot 2: Detección (Tu verificación humana)
    plt.subplot(1, 3, 2)
    plt.title("2. Detección de ROI")
    plt.imshow(img_con_caja)
    plt.axis('off')

    # Subplot 3: Input final a la IA
    plt.subplot(1, 3, 3)
    plt.title("3. Input MobileNet (224x224)")
    plt.imshow(roi_final)
    plt.axis('off')

    plt.show()


In [ ]:
# --- EJECUTAR ---
normalizar_hoja('test1.jpeg')

Error: No se pudo cargar la imagen.
